In [1]:
# Author: Davide Aloi
# Function: the script cleans the resting state data acquired at the Welly.
# Preprocessing steps applied: low / high pass filter, confound regression, 
# drift removal (detrend), standardisation (i.e. set mean to 0 and std to 1)
# (nb that's why the cleaned scans don't look like brains anymore,
# see https://neurostars.org/t/using-nilearn-image-clean-img/2413) and finally smoothing.
# 
# Reference: https://carpentries-incubator.github.io/SDC-BIDS-fMRI/05-data-cleaning-with-nilearn/index.html

import numpy as np
import glob
import os
from nilearn import image
from nilearn import signal

# Path to files to clean and smooth
# My confounds are the movement regressors + stick regressor (7 regressors, in file regr.txt) 
folder = 'D:\\Raindrop_data\\p01\\'
rs_files = glob.glob(folder + "*\\*\\fmri_data\\RESTING*\\nifti\\rR*.nii")
rp_files = glob.glob(folder + "*\\*\\fmri_data\\RESTING*\\nifti\\regr*.txt") # confounds

# Nilearn clean_img settings 
v_to_skip = 4 # number of initial volumes (and timepoints in the confound variable) to be skipped
t_r = 1.77
high_pass = 0.001
low_pass = 0.1

# Smoothing settings
kmm = 5 # kernel mm

c:\Users\davide\miniconda3\envs\neuroimg\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
for rs_scan, rp in zip(rs_files, rp_files):
    path = '\\'.join(rs_scan.split('\\')[0:-1]) # where to save the cleaned file
    fname = rs_scan.split('\\')[-1] # filename
    print(fname)

    # Loading data and confounds, and dropping first 4 volumes and confounds timepoints
    rs_data = image.load_img(rs_scan)
    rs_data = rs_data.slicer[:,:,:,v_to_skip:]
    confounds = np.loadtxt(rp, dtype=str, delimiter=',', skiprows=0)[v_to_skip:,:] 

    # Cleaning data
    rs_cleaned = image.clean_img(rs_data, confounds=confounds, detrend=True, standardize=True,
                                 low_pass=low_pass, high_pass=high_pass, t_r=t_r)
    # Smoothing
    rs_smoothed = image.smooth_img(rs_cleaned, fwhm=kmm) 

    # Saving
    rs_smoothed.to_filename(path + '\\s' + fname)

rRESTING_STATE_BASELINE_0017_resting_state_baseline_20220503122456_17a.nii


c:\Users\davide\miniconda3\envs\neuroimg\lib\site-packages\nilearn\datasets\__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
c:\Users\davide\miniconda3\envs\neuroimg\lib\site-packages\nilearn\image\image.py:1106: FutureWarning: The parameter "sessions" will be removed in 0.9.0 release of Nilearn. Please use the parameter "runs" instead.
  data = signal.clean(
